In [8]:

%pip install xlrd==1.2.0

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: C:\Users\dustn\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


주간 데이터 크롤링

In [4]:
import os
import time
from datetime import datetime, timedelta
from selenium import webdriver
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
import win32com.client as win32
from webdriver_manager.chrome import ChromeDriverManager

# 다운로드 경로 설정
download_dir = os.path.abspath("downloads")
if not os.path.exists(download_dir):
    os.makedirs(download_dir)

# Chrome 옵션 설정
options = webdriver.ChromeOptions()
prefs = {
    "download.default_directory": download_dir,  # 다운로드 경로
    "download.prompt_for_download": False,       # 다운로드 대화 상자 비활성화
    "safebrowsing.enabled": True                 # 안전 브라우징 활성화
}
options.add_experimental_option("prefs", prefs)

# 웹드라이버 설정
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)
driver.get("https://www.kobis.or.kr/kobis/business/stat/boxs/findWeeklyBoxOfficeList.do")

# WebDriverWait 설정
wait = WebDriverWait(driver, 10)

# '주간' 옵션 선택
weekly_option = wait.until(EC.presence_of_element_located((By.ID, "sWeekGb")))
select = Select(weekly_option)
select.select_by_visible_text("주간")  # 주간 옵션 선택

def set_datepicker(datepicker_id, year, month, day):
    # 데이터피커 열기
    date_picker = wait.until(EC.element_to_be_clickable((By.XPATH, f"//input[@id='{datepicker_id}']")))
    date_picker.click()

    # 년도 선택
    year_dropdown = wait.until(EC.element_to_be_clickable((By.XPATH, "//select[@class='ui-datepicker-year']")))
    year_dropdown.click()
    year_option = driver.find_element(By.XPATH, f"//option[text()='{year}']")
    year_option.click()

    # 월 선택
    month_dropdown = driver.find_element(By.XPATH, "//select[@class='ui-datepicker-month']")
    month_dropdown.click()
    month_option = driver.find_element(By.XPATH, f"//option[text()='{month}월']")  # 0부터 시작
    month_option.click()

    # 날짜 선택
    day_element = driver.find_element(By.XPATH, f"//a[text()='{day}']")
    day_element.click()

# 시작 날짜 설정
def set_start_and_end_dates(start_date):
    set_datepicker('sSearchFrom', start_date.year, start_date.month, start_date.day)

# 엑셀 파일 다운로드 및 저장
def download_and_rename_excel(start_date, end_date):
    # 조회 버튼 클릭
    search_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[@onclick=\"chkform('search');\"]")))
    search_button.click()

    # 엑셀 다운로드 버튼 클릭
    excel_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//a[@onclick=\"chkform('excel'); return false;\"]")))
    excel_button.click()

    # 알럿 창 대기 및 확인
    WebDriverWait(driver, 10).until(EC.alert_is_present())
    alert = Alert(driver)
    alert.accept()  # '확인' 클릭

    # 다운로드 후 파일 이름 변경
    time.sleep(7)  # 파일 다운로드 대기 (네트워크 환경에 따라 조정 필요)
    download_dir = os.path.abspath("downloads")
    original_file = os.path.join(download_dir, f"KOBIS_주간_주말_박스오피스_{datetime.today().strftime('%Y-%m-%d')}.xls")
    rename_file = os.path.join(download_dir, f"KOBIS_주간별박스오피스_{start_date.strftime('%Y%m%d')}-{end_date.strftime('%Y%m%d')}.xls")
    
    new_dir = os.path.abspath("excel")
    new_file = os.path.join(new_dir, f"KOBIS_주간별박스오피스_{start_date.strftime('%Y%m%d')}-{end_date.strftime('%Y%m%d')}.xlsx")

    if os.path.exists(original_file):
        os.rename(original_file, rename_file)
        print(f"파일 저장 완료: {rename_file}")
    else:
        print(f"파일을 찾을 수 없습니다: {original_file}")
        return
    
    # 파일 변환
    excel = None  # excel 객체 초기화
    if os.path.exists(rename_file):
        try:
            excel = win32.gencache.EnsureDispatch('Excel.Application')
            wb = excel.Workbooks.Open(rename_file)  # rename_file 경로 사용
            wb.SaveAs(new_file, FileFormat=51)  # 51은 XLSX 형식
            wb.Close()
            print(f"파일 저장 완료: {new_file}")
        except Exception as e:
            print(f"Excel 변환 중 오류 발생: {e}")
        finally:
            if excel is not None:  # excel 객체가 None이 아닌 경우에만 종료
                excel.Application.Quit()
    else:
        print(f"파일을 찾을 수 없습니다: {rename_file}")


# 날짜 범위 설정
start_date = datetime(2020, 1, 1)
end_date = datetime(2024, 12, 31)

# 주간 단위로 반복
while start_date <= end_date:
    current_end_date = start_date + timedelta(weeks=4 - 1)
    set_start_and_end_dates(start_date)
    download_and_rename_excel(start_date, current_end_date)

    # 다음 주기로 이동
    start_date = current_end_date + timedelta(days=1)

print(f"Files processed and saved to: {download_dir}")


파일 저장 완료: c:\study\de-dev-project2\downloads\KOBIS_주간별박스오피스_20200101-20200122.xls
파일 저장 완료: c:\study\de-dev-project2\excel\KOBIS_주간별박스오피스_20200101-20200122.xlsx
파일 저장 완료: c:\study\de-dev-project2\downloads\KOBIS_주간별박스오피스_20200123-20200213.xls
파일 저장 완료: c:\study\de-dev-project2\excel\KOBIS_주간별박스오피스_20200123-20200213.xlsx
파일 저장 완료: c:\study\de-dev-project2\downloads\KOBIS_주간별박스오피스_20200214-20200306.xls
파일 저장 완료: c:\study\de-dev-project2\excel\KOBIS_주간별박스오피스_20200214-20200306.xlsx
파일 저장 완료: c:\study\de-dev-project2\downloads\KOBIS_주간별박스오피스_20200307-20200328.xls
파일 저장 완료: c:\study\de-dev-project2\excel\KOBIS_주간별박스오피스_20200307-20200328.xlsx
파일 저장 완료: c:\study\de-dev-project2\downloads\KOBIS_주간별박스오피스_20200329-20200419.xls
파일 저장 완료: c:\study\de-dev-project2\excel\KOBIS_주간별박스오피스_20200329-20200419.xlsx
파일 저장 완료: c:\study\de-dev-project2\downloads\KOBIS_주간별박스오피스_20200420-20200511.xls
파일 저장 완료: c:\study\de-dev-project2\excel\KOBIS_주간별박스오피스_20200420-20200511.xlsx
파일 저장 완료: c:\study\de-dev-project2

In [ ]:
월간 데이터 크롤링

In [11]:
import os
import time
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from selenium import webdriver
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
import win32com.client as win32
from webdriver_manager.chrome import ChromeDriverManager

# 다운로드 경로 설정
download_dir = os.path.abspath("downloads")
if not os.path.exists(download_dir):
    os.makedirs(download_dir)

# Chrome 옵션 설정
options = webdriver.ChromeOptions()
prefs = {
    "download.default_directory": download_dir,  # 다운로드 경로
    "download.prompt_for_download": False,       # 다운로드 대화 상자 비활성화
    "safebrowsing.enabled": True                 # 안전 브라우징 활성화
}
options.add_experimental_option("prefs", prefs)

# 웹드라이버 설정
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)
driver.get("https://www.kobis.or.kr/kobis/business/stat/boxs/findMonthlyBoxOfficeList.do")

# WebDriverWait 설정
wait = WebDriverWait(driver, 10)

def set_datepicker(year, month):
    year_option = wait.until(EC.presence_of_element_located((By.ID, "sSearchYearFrom")))
    year_select = Select(year_option)
    year_select.select_by_visible_text(str(year))

    month_option = wait.until(EC.presence_of_element_located((By.ID, "sSearchMonthFrom")))
    month_select = Select(month_option)
    month_select.select_by_visible_text(str(month).zfill(2))

# 엑셀 파일 다운로드 및 저장
def download_and_rename_excel(start_date, end_date):
    # 조회 버튼 클릭
    search_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[@onclick=\"chkform('search');\"]")))
    search_button.click()

    # 엑셀 다운로드 버튼 클릭
    excel_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//a[@onclick=\"chkform('excel'); return false;\"]")))
    excel_button.click()

    # 알럿 창 대기 및 확인
    WebDriverWait(driver, 10).until(EC.alert_is_present())
    alert = Alert(driver)
    alert.accept()  # '확인' 클릭

    # 다운로드 후 파일 이름 변경
    time.sleep(7)  # 파일 다운로드 대기 (네트워크 환경에 따라 조정 필요)
    download_dir = os.path.abspath("downloads")
    original_file = os.path.join(download_dir, f"KOBIS_월별박스오피스_{datetime.today().strftime('%Y-%m-%d')}.xls")
    rename_file = os.path.join(download_dir, f"KOBIS_월별박스오피스_{start_date.strftime('%Y-%m')}-{end_date.strftime('%m')}.xls")
    
    new_dir = os.path.abspath("excel")
    new_file = os.path.join(new_dir, f"KOBIS_월별박스오피스_{start_date.strftime('%Y-%m')}-{end_date.strftime('%m')}.xlsx")

    if os.path.exists(original_file):
        os.rename(original_file, rename_file)
        print(f"파일 저장 완료: {rename_file}")
    else:
        print(f"파일을 찾을 수 없습니다: {original_file}")
        return
    
    # 파일 변환
    excel = None  # excel 객체 초기화
    if os.path.exists(rename_file):
        try:
            excel = win32.gencache.EnsureDispatch('Excel.Application')
            wb = excel.Workbooks.Open(rename_file)  # rename_file 경로 사용
            wb.SaveAs(new_file, FileFormat=51)  # 51은 XLSX 형식
            wb.Close()
            print(f"파일 저장 완료: {new_file}")
        except Exception as e:
            print(f"Excel 변환 중 오류 발생: {e}")
        finally:
            if excel is not None:  # excel 객체가 None이 아닌 경우에만 종료
                excel.Application.Quit()
    else:
        print(f"파일을 찾을 수 없습니다: {rename_file}")


# 날짜 범위 설정
start_date = datetime(2020, 1, 1)
end_date = datetime(2024, 7, 1)

# 6개월 단위로 반복
while start_date <= end_date:
    current_end_date = start_date + relativedelta(months=5)
    set_datepicker(start_date.year, start_date.month)
    download_and_rename_excel(start_date, current_end_date)

    # 다음 주기로 이동
    start_date = current_end_date + relativedelta(months=1)

print(f"Files processed and saved to: {download_dir}")


파일 저장 완료: c:\study\de-dev-project2-git\downloads\KOBIS_월별박스오피스_2020-01-06.xls
파일 저장 완료: c:\study\de-dev-project2-git\excel\KOBIS_월별박스오피스_2020-01-06.xlsx
파일 저장 완료: c:\study\de-dev-project2-git\downloads\KOBIS_월별박스오피스_2020-07-12.xls
파일 저장 완료: c:\study\de-dev-project2-git\excel\KOBIS_월별박스오피스_2020-07-12.xlsx
파일 저장 완료: c:\study\de-dev-project2-git\downloads\KOBIS_월별박스오피스_2021-01-06.xls
파일 저장 완료: c:\study\de-dev-project2-git\excel\KOBIS_월별박스오피스_2021-01-06.xlsx
파일 저장 완료: c:\study\de-dev-project2-git\downloads\KOBIS_월별박스오피스_2021-07-12.xls
파일 저장 완료: c:\study\de-dev-project2-git\excel\KOBIS_월별박스오피스_2021-07-12.xlsx
파일 저장 완료: c:\study\de-dev-project2-git\downloads\KOBIS_월별박스오피스_2022-01-06.xls
파일 저장 완료: c:\study\de-dev-project2-git\excel\KOBIS_월별박스오피스_2022-01-06.xlsx
파일 저장 완료: c:\study\de-dev-project2-git\downloads\KOBIS_월별박스오피스_2022-07-12.xls
파일 저장 완료: c:\study\de-dev-project2-git\excel\KOBIS_월별박스오피스_2022-07-12.xlsx
파일 저장 완료: c:\study\de-dev-project2-git\downloads\KOBIS_월별박스오피스_2023-01-06.xls
파일 저

주간 데이터 csv 생성

In [19]:
import pandas as pd
import os
import re
from datetime import datetime, timedelta

# 여러 엑셀 파일이 저장된 디렉토리 경로
input_folder = "excel"
output_csv_file = "csv/weekly_combined_data.csv"

# 월별 데이터 구분을 위한 변수 초기화
all_data = []

# 파일 처리
for file_name in os.listdir(input_folder):
    if file_name.startswith("KOBIS_주간") and file_name.endswith(".xlsx"):  # 파일명 조건
        file_path = os.path.join(input_folder, file_name)
        print(f"처리 중인 파일: {file_path}")
        
        # 엑셀 파일 읽기
        df = pd.read_excel(file_path, skiprows=4)

        # 데이터 처리
        current_week_data = []
        pattern = re.compile(r".*\((\d{4}년 \d{2}월 \d{2}일) ~ .*")
        start_date = ""
        end_date = ""
        
        for idx, row in df.iterrows():
            row_values = row.tolist()  # 현재 행에서 NaN 값을 제거한 배열
            
            if isinstance(row_values[0], str): 
                match = pattern.match(row_values[0])
                if match: # 제목행
                    original_date = match.group(1)
                    start_date = datetime.strptime(original_date, "%Y년 %m월 %d일")
                    end_date = start_date + timedelta(days=6)
                    
                    if current_week_data:  # 이전 데이터가 있으면 저장
                        month_df = pd.DataFrame(current_week_data)
                        all_data.append(month_df)
                    
                    current_week_data = []
            elif isinstance(row_values[0], int): # 일반 데이터 행               
                row_values.extend(['week', start_date.strftime("%Y-%m-%d"), end_date.strftime("%Y-%m-%d")])
                current_week_data.append(row_values)
        else:
            if current_week_data:
                month_df = pd.DataFrame(current_week_data)
                all_data.append(month_df)

# 모든 주간 데이터를 하나의 DataFrame으로 병합
if all_data:  # 데이터가 있을 경우 병합
    final_df = pd.concat(all_data, ignore_index=True)
    final_df.columns = [
        "rank", "movie_name", "release_date", "sales", "sales_share", "sales_change", "sales_change_rate",
        "cumulative_sales", "audience", "audience_change", "audience_change_rate", "cumulative_audience", 
        "screen_count", "show_count", "main_country", "country", 
        "distributor", "period_type", "period_start_date", "period_end_date"
    ]

    final_df.to_csv(output_csv_file, index=False, encoding='utf-8-sig')
    print(f"CSV 파일 생성 완료: {output_csv_file}")
else:
    print("병합할 데이터가 없습니다.")


처리 중인 파일: excel\KOBIS_주간별박스오피스_20200101-20200122.xlsx
처리 중인 파일: excel\KOBIS_주간별박스오피스_20200123-20200213.xlsx
처리 중인 파일: excel\KOBIS_주간별박스오피스_20200214-20200306.xlsx
처리 중인 파일: excel\KOBIS_주간별박스오피스_20200307-20200328.xlsx
처리 중인 파일: excel\KOBIS_주간별박스오피스_20200329-20200419.xlsx
처리 중인 파일: excel\KOBIS_주간별박스오피스_20200420-20200511.xlsx
처리 중인 파일: excel\KOBIS_주간별박스오피스_20200512-20200602.xlsx
처리 중인 파일: excel\KOBIS_주간별박스오피스_20200603-20200624.xlsx
처리 중인 파일: excel\KOBIS_주간별박스오피스_20200625-20200716.xlsx
처리 중인 파일: excel\KOBIS_주간별박스오피스_20200717-20200807.xlsx
처리 중인 파일: excel\KOBIS_주간별박스오피스_20200808-20200829.xlsx
처리 중인 파일: excel\KOBIS_주간별박스오피스_20200830-20200920.xlsx
처리 중인 파일: excel\KOBIS_주간별박스오피스_20200921-20201012.xlsx
처리 중인 파일: excel\KOBIS_주간별박스오피스_20201013-20201103.xlsx
처리 중인 파일: excel\KOBIS_주간별박스오피스_20201104-20201125.xlsx
처리 중인 파일: excel\KOBIS_주간별박스오피스_20201126-20201217.xlsx
처리 중인 파일: excel\KOBIS_주간별박스오피스_20201218-20210108.xlsx
처리 중인 파일: excel\KOBIS_주간별박스오피스_20210109-20210130.xlsx
처리 중인 파일: excel\KOBIS_주간별박스오

연간 데이터 csv 생성

In [2]:
import pandas as pd
import os

# 여러 엑셀 파일이 저장된 디렉토리 경로
input_folder = "excel"
output_csv_file = "csv/year_combined_data.csv"

# 엑셀 파일을 하나로 합치기 위한 빈 리스트 생성
combined_data = []

# 디렉토리에서 모든 엑셀 파일 읽어오기
for file_name in os.listdir(input_folder):
    if file_name.startswith("KOBIS_연도별") and file_name.endswith(".xlsx"):  # 엑셀 파일만 선택
        file_path = os.path.join(input_folder, file_name)
        
        # 엑셀 파일 로드
        df = pd.read_excel(file_path, skiprows=4)

        # 컬럼 이름 재정의
        df.columns = ["rank", "movie_name", "release_date", "sales", "sales_share", "audience", "screen_count", "show_count", "main_country", "country", "distributor"]

        # 합계 행 제거
        df = df.iloc[:-1]

        # 파일 이름에서 연도 추출 (예: _2024)
        parts = file_name.split('_')  # 파일 이름을 '_'로 분할
        file_date = parts[-1].split('.')[0] if len(parts) > 1 else "Unknown"  # 마지막 부분에서 확장자를 제거
        
        
        # 추가 컬럼 (출처 구분용)
        df['period_type'] = 'year'  
        df['period_start_date'] = file_date 
        
        # 리스트에 추가
        combined_data.append(df)

# 모든 데이터를 하나의 DataFrame으로 합치기
final_df = pd.concat(combined_data, ignore_index=True)

# 합친 데이터를 CSV로 저장
final_df.to_csv(output_csv_file, index=False, encoding='utf-8-sig')
print(f"CSV 파일 생성 완료: {output_csv_file}")


CSV 파일 생성 완료: csv/year_combined_data.csv


월간 데이터 CSV 생성

In [12]:
import pandas as pd
import os

# 여러 엑셀 파일이 저장된 디렉토리 경로
input_folder = "excel"
output_csv_file = "csv/month_combined_data.csv"

# 월별 데이터 구분을 위한 변수 초기화
all_data = []

# 파일 처리
for file_name in os.listdir(input_folder):
    if file_name.startswith("KOBIS_월별") and file_name.endswith(".xlsx"):  # 파일명 조건
        file_path = os.path.join(input_folder, file_name)
        print(f"처리 중인 파일: {file_path}")
        
        try:
            year, start_month, end_month = map(int, file_name.split("_")[-1].split(".")[0].split("-"))
        except ValueError:
            print(f"파일명에서 기간을 추출할 수 없습니다: {file_name}")
            continue
        
        # 엑셀 파일 읽기
        df = pd.read_excel(file_path, skiprows=4)

        # 데이터 처리
        current_month = None
        current_month_data = []
        month_num = 0
        
        for idx, row in df.iterrows():
            row_values = row.tolist()  # 현재 행에서 NaN 값을 제거한 배열
            
            if isinstance(row_values[0], str) and '월' in row_values[0]:
                # 현재 월 갱신 및 새로운 데이터 초기화
                current_month = ''.join(filter(str.isdigit, row_values[0]))[4:]
                month_num = int(current_month)

                if current_month_data:  # 이전 월 데이터가 있으면 저장
                    month_df = pd.DataFrame(current_month_data)
                    all_data.append(month_df)
                
                current_month_data = []
            elif isinstance(row_values[0], int):    # 일반 데이터 행   
                if month_num >= start_month and month_num <= end_month: # 7월 데이터는 제외
                    if pd.isna(row_values[8]):
                        row_values[8] = 0

                    row_values.extend(['month', f"{year}-{str(month_num).zfill(2)}"])
                    current_month_data.append(row_values)
        else:
            if current_month_data:  # 이전 월 데이터가 있으면 저장
                    # 7월 데이터는 제외
                    if month_num >= start_month and month_num <= end_month:
                        month_df = pd.DataFrame(current_month_data)
                        all_data.append(month_df)

# 모든 월 데이터를 하나의 DataFrame으로 병합
if all_data:  # 데이터가 있을 경우 병합
    final_df = pd.concat(all_data, ignore_index=True)
    final_df.columns = [
        "rank", "movie_name", "release_date", "sales", "sales_share", 
        "cumulative_sales", "audience", "cumulative_audience", 
        "screen_count", "show_count", "main_country", "country", 
        "distributor", "period_type", "period_start_date"
    ]

    final_df.to_csv(output_csv_file, index=False, encoding='utf-8-sig')
    print(f"CSV 파일 생성 완료: {output_csv_file}")
else:
    print("병합할 데이터가 없습니다.")


처리 중인 파일: excel\KOBIS_월별박스오피스_2020-01-06.xlsx
처리 중인 파일: excel\KOBIS_월별박스오피스_2020-07-12.xlsx
처리 중인 파일: excel\KOBIS_월별박스오피스_2021-01-06.xlsx
처리 중인 파일: excel\KOBIS_월별박스오피스_2021-07-12.xlsx
처리 중인 파일: excel\KOBIS_월별박스오피스_2022-01-06.xlsx
처리 중인 파일: excel\KOBIS_월별박스오피스_2022-07-12.xlsx
처리 중인 파일: excel\KOBIS_월별박스오피스_2023-01-06.xlsx
처리 중인 파일: excel\KOBIS_월별박스오피스_2023-07-12.xlsx
처리 중인 파일: excel\KOBIS_월별박스오피스_2024-01-06.xlsx
처리 중인 파일: excel\KOBIS_월별박스오피스_2024-07-12.xlsx
CSV 파일 생성 완료: month_combined_data.csv
